## Deriving covariates across the northern bobwhite range


In [1]:
from datetime import date
today = date.today()
import ee
import math
import geemap
import numpy as np
import pandas as pd
import os


# Initialize ee and authenticate 
#ee.Authenticate()
ee.Initialize()



# Important Information

## Script name: 

## Purpose of script:
This is a preliminary script for deriving covariates across the northern bobwhite range for use in the construction of a species distribution model.
## Author: 
Patrick Freeman (CSP)
## Date Created: 
03/15/23
## Date last modified:
print('Last Updated On: ', datetime.datetime.now())
## Email: 
patrick[at]csp-inc.org
## ---------------------------
## Notes:

## ---------------------------

In [ ]:
#install geemap module as needed
#!pip install geemap 

In [1]:
import os
import ee
import geemap

In [2]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


# Write utility functions

In [2]:
# FUNCTIONS

# Focal mean
def focal_mean(image, radius, unit, name):
    names = image.bandNames().getInfo()
    new_names = [s + name for s in names]
    return image.reduceNeighborhood(kernel = ee.Kernel.circle(radius, unit),
                                    reducer = ee.Reducer.mean()).rename(new_names)

# Focal median
def focal_median(image, radius, unit):
    return image.reduceNeighborhood(kernel = ee.Kernel.circle(radius, unit),
                                    reducer = ee.Reducer.median())
    
# Focal SD
def focal_sd(image, radius, unit):
    return image.reduceNeighborhood(kernel = ee.Kernel.circle(radius, unit),
                                    reducer = ee.Reducer.stdDev())

# Focal sum
def focal_sum(image, radius, unit):
    return image.reduceNeighborhood(kernel = ee.Kernel.circle(radius, unit, False),
                                    reducer = ee.Reducer.sum())

# Focal count
def focal_count(image, radius, unit):
    return image.reduceNeighborhood(kernel = ee.Kernel.circle(radius, unit, False),
                                    reducer = ee.Reducer.count())
    
# Percent cover
def percent_cov(image, radius, unit, name):
    names = image.bandNames().getInfo()
    new_names = [s + name for s in names]
    isum = focal_sum(image, radius, unit)
    icount = focal_count(image, radius, unit)
    return isum.divide(icount).rename(new_names)

def toFloat(img):
    return img.float()

# Set aoi, spatial scale and projection of export, and smoothing parameters

In [3]:
## Bring in buffered range map as 'region' 
region = ee.FeatureCollection('projects/GEE_CSP/pf-bobwhite/bobwhite_model_states')
geometry = ee.Feature(ee.FeatureCollection(region).first())
conus_geom = ee.FeatureCollection("projects/GEE_CSP/thirty-by-thirty/aoi_conus")
conus_img = ee.Image("projects/GEE_CSP/thirty-by-thirty/aoi_conus_mask")

# export scale and projection
scale = 250
projection = ee.Projection('EPSG:5070') # stand-in for now. Figure out best projection to use 

# Choose radii for summarizing covariates
rad_large = 10000
rad_small = 5000
name_large = "_10km"
name_small = "_5km"

## Plot to check 
#Map = geemap.Map(center=(40, -100), zoom=4)
#Map.addLayer(geometry, {}, "Model states", True)
#Map


# CSP-derived land use intensity layers related to agriculture, transportation, urban development, and energy infrastructure

In [4]:
lui = ee.Image("projects/GEE_CSP/aft-connectivity/Land-use-intensity-multiband-focal-sp-250m-20220123")

### Get the band names as a check 
lui_names = lui.bandNames()
print('lui Band names:', lui_names.getInfo())  # ee.List of band names
lui_focal_means_large = focal_mean(lui, rad_large, "meters", name_large).updateMask(conus_img).clip(geometry)
lui_focal_means_small = focal_mean(lui, rad_small, "meters", name_small).updateMask(conus_img).clip(geometry)

### Get the band names as a check 
lui_focal_means_names = lui_focal_means_large.bandNames()
print('lui_focal_means Band names:', lui_focal_means_names.getInfo())  # ee.List of band names

lui Band names: ['Ag', 'Urban', 'Transport', 'Energy']
lui_focal_means Band names: ['Ag_10km', 'Urban_10km', 'Transport_10km', 'Energy_10km']


# RAP Proportional Cover

In [5]:
geometry = ee.Feature(ee.FeatureCollection("projects/GEE_CSP/pf-bobwhite/bobwhite_model_states").first());
##---------- Define the years that you want to export --------------
##---------- End year is inclusive in this case  ------------------
yearStart = 2016
yearEnd = 2021

## -------------- Define the plant functional types (PFTs) that you want to export --------------
## PFTs are "AFGC" (Annual forb and grass cover), "BG" (bare ground), "LTR" (litter), 
## "PFGC" (perennial forb and grass cover), "SHR" (shrub cover), and "TREE" (tree cover)
## Select Annual forb and grass cover, perennial forb and grass cover, shrub cover, and tree cover 
PFTs = ee.List(['AFG', 'PFG', 'SHR', 'TRE', 'BGR']);

cover = ee.ImageCollection("projects/rangeland-analysis-platform/vegetation-cover-v3")
## ------------- Select the PFTs for processing as defined by User  --------------
cover_toExport = cover.select(PFTs)


### Filter RAP cover ImageCollection into yearly sets
rap_cover_2016 = cover_toExport.filter(ee.Filter.inList('year', ee.List([2016]))).toBands()
rap_cover_2017 = cover_toExport.filter(ee.Filter.inList('year', ee.List([2017]))).toBands()
rap_cover_2018 = cover_toExport.filter(ee.Filter.inList('year', ee.List([2018]))).toBands()
rap_cover_2019 = cover_toExport.filter(ee.Filter.inList('year', ee.List([2019]))).toBands()
rap_cover_2021 = cover_toExport.filter(ee.Filter.inList('year', ee.List([2021]))).toBands()

### Combine all into single multiband image 
rap_cover_all = ee.Image([rap_cover_2016, rap_cover_2017, rap_cover_2018, rap_cover_2019, rap_cover_2021])

rap_cover_all_band_names = rap_cover_all.bandNames()
print('rap_cover_2016 Band names:', rap_cover_all_band_names.getInfo())  # ee.List of band names

### Apply focal mean smoothing
rap_cover_all_small = focal_mean(rap_cover_all, rad_small, "meters", name_small).clip(geometry)
rap_cover_all_small_band_names = rap_cover_all_small.bandNames()
print('rap_cover_all_small Band names:', rap_cover_all_small_band_names.getInfo())  # ee.List of band names



rap_cover_2016 Band names: ['2016_AFG', '2016_PFG', '2016_SHR', '2016_TRE', '2016_BGR', '2017_AFG', '2017_PFG', '2017_SHR', '2017_TRE', '2017_BGR', '2018_AFG', '2018_PFG', '2018_SHR', '2018_TRE', '2018_BGR', '2019_AFG', '2019_PFG', '2019_SHR', '2019_TRE', '2019_BGR', '2021_AFG', '2021_PFG', '2021_SHR', '2021_TRE', '2021_BGR']
rap_cover_all_small Band names: ['2016_AFG_5km', '2016_PFG_5km', '2016_SHR_5km', '2016_TRE_5km', '2016_BGR_5km', '2017_AFG_5km', '2017_PFG_5km', '2017_SHR_5km', '2017_TRE_5km', '2017_BGR_5km', '2018_AFG_5km', '2018_PFG_5km', '2018_SHR_5km', '2018_TRE_5km', '2018_BGR_5km', '2019_AFG_5km', '2019_PFG_5km', '2019_SHR_5km', '2019_TRE_5km', '2019_BGR_5km', '2021_AFG_5km', '2021_PFG_5km', '2021_SHR_5km', '2021_TRE_5km', '2021_BGR_5km']


# Climate covariates from Daymet

In [6]:
### Load daymet dataset 
daymet_17 = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filter(ee.Filter.date('2017-01-01', '2017-12-31'))
daymet_18 = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filter(ee.Filter.date('2018-01-01', '2018-12-31'))
daymet_19 = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filter(ee.Filter.date('2019-01-01', '2019-12-31'))
daymet_21 = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filter(ee.Filter.date('2021-01-01', '2021-12-31'))


### mean daily max temperature
tmax_17 = daymet_17.select("tmax").mean().clip(geometry).rename(['tmax_17'])
tmax_18 = daymet_18.select("tmax").mean().clip(geometry).rename(['tmax_18'])
tmax_19 = daymet_19.select("tmax").mean().clip(geometry).rename(['tmax_19'])
tmax_21 = daymet_21.select("tmax").mean().clip(geometry).rename(['tmax_21'])

### mean daily min temperature
tmin_17 = daymet_17.select("tmin").mean().clip(geometry).rename(['tmin_17'])
tmin_18 = daymet_18.select("tmin").mean().clip(geometry).rename(['tmin_18'])
tmin_19 = daymet_19.select("tmin").mean().clip(geometry).rename(['tmin_19'])
tmin_21 = daymet_21.select("tmin").mean().clip(geometry).rename(['tmin_21'])

### mean daily precip
prcp_17 = daymet_17.select("prcp").mean().clip(geometry).rename(['prcp_17'])
prcp_18 = daymet_18.select("prcp").mean().clip(geometry).rename(['prcp_18'])
prcp_19 = daymet_19.select("prcp").mean().clip(geometry).rename(['prcp_19'])
prcp_21 = daymet_21.select("prcp").mean().clip(geometry).rename(['prcp_21'])

### mean daily snow water equivalent
swe_17 = daymet_17.select("swe").mean().clip(geometry).rename(['swe_17'])
swe_18 = daymet_18.select("swe").mean().clip(geometry).rename(['swe_18'])
swe_19 = daymet_19.select("swe").mean().clip(geometry).rename(['swe_19'])
swe_21 = daymet_21.select("swe").mean().clip(geometry).rename(['swe_21'])

climate_all = ee.Image([tmax_17, tmax_18, tmax_19, tmax_21, tmin_17, tmin_18, tmin_19, tmin_21, 
                        prcp_17, prcp_18, prcp_19, prcp_21, swe_17, swe_18, swe_19, swe_21])

climate_small = focal_mean(climate_all, rad_small, "meters", name_small).updateMask(conus_img).clip(geometry)



# SNODAS Snow Depth Data

In [16]:
snodas_17 = ee.Image("projects/GEE_CSP/pf-bobwhite/SNODAS/snodas_days_snowdepth_2017_WGS84").rename('snodays_17')
snodas_18 = ee.Image("projects/GEE_CSP/pf-bobwhite/SNODAS/snodas_days_snowdepth_2018_WGS84").rename('snodays_18')
snodas_19 = ee.Image("projects/GEE_CSP/pf-bobwhite/SNODAS/snodas_days_snowdepth_2019_WGS84").rename('snodays_19')
snodas_21 = ee.Image("projects/GEE_CSP/pf-bobwhite/SNODAS/snodas_days_snowdepth_2021_WGS84").rename('snodays_21')

snodas_all = ee.Image([snodas_17, snodas_18, snodas_19, snodas_21])

snodas_small = focal_mean(snodas_all, rad_small, "meters", name_small).clip(geometry)
snodas_small_band_names = snodas_small.bandNames()
print('Snodas data band names:', snodas_small_band_names.getInfo())  # ee.List of band names


Snodas data band names: ['snodays_17_5km', 'snodays_18_5km', 'snodays_19_5km', 'snodays_21_5km']


# NASS and CRP Stats on Farm Size and Proportion of County Land in CRP categories positive for bobwhite

In [17]:
avg_farm_size = ee.Image("projects/GEE_CSP/pf-bobwhite/NASS_CRP/nass_2017_average_farm_size").rename('avg_farm_size_17')
prop_crp = ee.Image("projects/GEE_CSP/pf-bobwhite/NASS_CRP/proportion-county-land-crp-2020").rename('prop_crp_acres_20')

nass_crp_all = ee.Image([avg_farm_size, prop_crp])

# Bring all covariates together into a single projection

In [18]:
lui_prep = lui_focal_means_small.reproject(crs=projection, scale=5000)
rap_prep = rap_cover_all_small.reproject(crs=projection, scale=5000)
climate_prep = climate_small.reproject(crs=projection, scale=5000)
snodas_prep = snodas_all.reproject(crs=projection, scale=5000)
nass_crp_prep = nass_crp_all.reproject(crs=projection,scale=5000)

covs = ee.Image([lui_prep, rap_prep, climate_prep, snodas_prep, nass_crp_prep])


In [ ]:

vis_params = {
    'bands': 'Ag_5km',
    'min': 0,
    'max': 1,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
## Plot to check 
Map = geemap.Map(center=(40, -100), zoom=4)
Map.addLayer(lui.prep, vis_params, "LUI reprojected", True)
Map.addLayer(lui_focal_means_small, vis_params, "LUI original", False)
Map


# Extract random point data for assessment of covariate correlation

In [19]:
# Extract local covariate values from multi-band predictor image at 5000 random points
randomPts = ee.FeatureCollection.randomPoints(region.geometry(), 5000)
work_dir = os.path.expanduser('~/Downloads')
print(work_dir)
out_csv = os.path.join(work_dir, 'randomPtsCovs.csv')
extraction = geemap.extract_values_to_points(randomPts, covs, out_csv)


/Users/patrickfreeman-csp/Downloads
Generating URL ...
Please wait ...
Data downloaded to /Users/patrickfreeman-csp/Downloads/randomPtsCovs.csv
